In [6]:
#biblio8ikes gia to kommati ton praksewn kai gia tous xartes
#import numpy as np
import import_ipynb
import folium
import ApoKomb1
import db1
import warnings
import mapGre
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import networkx as nx
#biblio8ikes gia to grafiko kommati
from PyQt5 import QtCore, QtGui, QtWidgets, QtWebEngineWidgets
import io
import sys

import torch



class basiko(QtWidgets.QMainWindow):       
        
    def main_program(self, num_points):
        #pernoume thn basi dedomenwn pou einai se exel kai thn dumiourgoume se pinaka
        citys = db1.basi()
        
        #edw analoga me to koumpi pou pataei o xristeis pernoume ton 
        #ari8mo dedomenon apo ton pinaka
        if num_points == 5:
            citys1 = citys.sample(frac=0.009)
        elif num_points == 10:
            citys1 = citys.sample(frac=0.017)
        elif num_points == 20:
            citys1 = citys.sample(frac=0.033)
        else:
            citys1 = citys.sample(frac=0.083)

           
        #edw xwrizoume ta dedomena analoga se Nodes Edges 
        node_citys = citys1[["city"]]

        lats_c = citys1[["lat"]]
        lngs_c = citys1[["lng"]]

        location = citys1[["lat", "lng"]].astype(float)
        coords = ApoKomb1.sindet(lats_c, lngs_c) 
     
        #coords = ApoKomb.sindet(lats_c, lngs_c)
        #distances = ApoKomb.apostasis(node_citys, coords)
        #print(distances) 
        node_citys =  np.array(citys1["city"])
        #edw pros8etoume sto map ollous tous kombous kai ths apostaseis metaksei tous 
        self.m = mapGre.mapMarker(self, lats_c, lngs_c, node_citys, coords)
        
        
        #Edw ta topo8etoume sto para8iro 
        data = io.BytesIO()
        self.m.save(data, close_file=False)
        self.view.setHtml(data.getvalue().decode(), QtCore.QUrl('file://'))
        
        return citys1
        
    
    def city_names_and_coords(c_sample):
        #citys = db1.basi()

        citys1 = c_sample
        #citys.sample(50)
        lats_c = citys1[["lat"]]
        lngs_c = citys1[["lng"]]
        node_citys =  np.array(citys1["city"])

        location = citys1[["lat", "lng"]].astype(float)
        #edw xrisimopoioume ena DataFrame gia na mporoume na exoume to ka8e edge se sigkekrimeno ar8imo gia na
        #mhn xa8oume sthn sunexia apo pio polu paei se pia 

        coords = ApoKomb1.sindet(lats_c, lngs_c) 
        edge_index = ApoKomb1.edge_index(node_citys)
        distances = ApoKomb1.apostasis(node_citys, coords)
        
        print("Oi polis pou exoume einai:", node_citys)
        return node_citys, coords, edge_index ,distances
    
    
    def map_path(self):
        
        #print("H beltisti diadromi einai:")
        #for i in range(len(self.node_citys)):
            #print(self.node_citys[self.best_solution[i]])
        self.m = mapGre.mapPath(self, self.node_citys, self.coords, self.best_solution)


        #Edw ta topo8etoume sto para8iro 
        data = io.BytesIO()
        self.m.save(data, close_file=False)
        self.view.setHtml(data.getvalue().decode(), QtCore.QUrl('file://'))

        return self.m        
      

#sto sigkekrimeno meros tou kwdika pernoume ta dedomena apo prin kai ta topo8etoume se enan algorithmo gia na 
#broume tis apostaseis apo ka8e kombo, sth synexia ta bazoume ta apotelesmata os (EDGE FEATURES) kai meta
#Ftiaxnoume ena grafima me auta kai sth sunexia to metatrepoume se gramiko grafima gia na exoume ta (EDGE FEATURES) os (NODE FEATURES)

# Example usage:
# Create a sample graph
class graph_to_LineGraph():
    
    def create_graph(edge_index, distances):
        G = nx.Graph()
        for i in range(len(edge_index)):
            G.add_edge(edge_index[i][0], edge_index[i][1], weight=distances[i])

        L = graph_to_LineGraph.line_graph(G)
        L,edge_index_lg, node_features = graph_to_LineGraph.complete_line_graph(L)
        
        node_features = torch.tensor(node_features, dtype=torch.float32).unsqueeze(1)


        return L, edge_index_lg, node_features    

    def line_graph(G):
        L = nx.line_graph(G)
        for node in L.nodes:
            edges = G.edges(node)
            if edges:
                edge_features = G.edges[node]['weight']
                L.nodes[node]['e_f'] = edge_features
            else:
                L.nodes[node]['e_f'] = None
        return L

    def line_graph_node_features(G):
        node_features = []
        for _, node_data in G.nodes(data=True):
            node_features.append(node_data['e_f'])
        return node_features

    def complete_line_graph(L):
        #L = line_graph(G)
        node_features1 = graph_to_LineGraph.line_graph_node_features(L)

        # Create a mapping from original graph edges to line graph nodes
        edge_to_index = {edge: idx for idx, edge in enumerate(L.nodes())}

        # Prepare edge index for the line graph
        edge_index_t = []
        for edge in L.edges:
            src, dst = edge
            edge_index_t.append([edge_to_index[src], edge_to_index[dst]])

        # Convert edge list to a 2D tensor
        edge_index_lg = torch.tensor(edge_index_t, dtype=torch.long).t().contiguous()
        #print(edge_index_lg)
        # Convert node features to tensor
        #nf = torch.tensor(node_features1, dtype=torch.float32).unsqueeze(1)


        #nx.draw(G, with_labels= True)
        # Transform the graph into its line graph representation
       # print('Original Graph: ',G)


        #nx.draw(L, with_labels= True)
       #print('Line Graph: ',L)
        return L, edge_index_lg, node_features1
        
        
   
    
    
 
 

In [ ]:
#biblio8ikes gia to kommati ton praksewn kai gia tous xartes
#import numpy as np
import import_ipynb
import folium
import ApoKomb1
import db1
import warnings
import mapGre
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import networkx as nx
#biblio8ikes gia to grafiko kommati
from PyQt5 import QtCore, QtGui, QtWidgets, QtWebEngineWidgets
import io
import sys

import torch



class basiko1(QtWidgets.QMainWindow):       
        
    def main_program(self, num_points):
        #pernoume thn basi dedomenwn pou einai se exel kai thn dumiourgoume se pinaka
        citys = db1.basi()
        
        #edw analoga me to koumpi pou pataei o xristeis pernoume ton 
        #ari8mo dedomenon apo ton pinaka
        if num_points == 5:
            citys1 = citys.sample(frac=0.009)
        elif num_points == 10:
            citys1 = citys.sample(frac=0.017)
        elif num_points == 20:
            citys1 = citys.sample(frac=0.033)
        else:
            citys1 = citys.sample(frac=0.083)

           
        #edw xwrizoume ta dedomena analoga se Nodes Edges 
        node_citys = citys1[["city"]]

        lats_c = citys1[["lat"]]
        lngs_c = citys1[["lng"]]

        location = citys1[["lat", "lng"]].astype(float)
        
     
        #coords = ApoKomb.sindet(lats_c, lngs_c)
        #distances = ApoKomb.apostasis(node_citys, coords)
        #print(distances) 
        
        #edw pros8etoume sto map ollous tous kombous kai ths apostaseis metaksei tous 
        m = mapGre.mapMarker(lats_c, lngs_c, citys1, location)
        
        
        #Edw ta topo8etoume sto para8iro 
        #data = io.BytesIO()
        #m.save(data, close_file=False)
        #self.view.setHtml(data.getvalue().decode(), QtCore.QUrl('file://'))
        
        return citys1, m
        
    
    def city_names_and_coords(c_sample):
        #citys = db1.basi()

        citys1 = c_sample
        #citys.sample(50)
        lats_c = citys1[["lat"]]
        lngs_c = citys1[["lng"]]
        node_citys =  np.array(citys1["city"])

        location = citys1[["lat", "lng"]].astype(float)
        #edw xrisimopoioume ena DataFrame gia na mporoume na exoume to ka8e edge se sigkekrimeno ar8imo gia na
        #mhn xa8oume sthn sunexia apo pio polu paei se pia 

        coords = ApoKomb1.sindet(lats_c, lngs_c) 
        edge_index = ApoKomb1.edge_index(node_citys)
        distances = ApoKomb1.apostasis(node_citys, coords)
        
        print("Oi polis pou exoume einai:", node_citys)
        return node_citys, coords, edge_index ,distances
    
    
    def map_path(self):
        
        print("H beltisti diadromi einai:")
        for i in range(len(self.node_citys)):
            print(self.node_citys[self.best_solution[i]])
        m = mapGre.mapPath(self.node_citys, self.coords, self.best_solution)


        #Edw ta topo8etoume sto para8iro 
        data = io.BytesIO()
        m.save(data, close_file=False)
        self.view.setHtml(data.getvalue().decode(), QtCore.QUrl('file://'))

        return m        
      

#sto sigkekrimeno meros tou kwdika pernoume ta dedomena apo prin kai ta topo8etoume se enan algorithmo gia na 
#broume tis apostaseis apo ka8e kombo, sth synexia ta bazoume ta apotelesmata os (EDGE FEATURES) kai meta
#Ftiaxnoume ena grafima me auta kai sth sunexia to metatrepoume se gramiko grafima gia na exoume ta (EDGE FEATURES) os (NODE FEATURES)

# Example usage:
# Create a sample graph
class graph_to_LineGraph():
    
    def create_graph(edge_index, distances):
        G = nx.Graph()
        for i in range(len(edge_index)):
            G.add_edge(edge_index[i][0], edge_index[i][1], weight=distances[i])

        L = graph_to_LineGraph.line_graph(G)
        L,edge_index_lg, node_features = graph_to_LineGraph.complete_line_graph(L)
        
        node_features = torch.tensor(node_features, dtype=torch.float32).unsqueeze(1)


        return L, edge_index_lg, node_features    

    def line_graph(G):
        L = nx.line_graph(G)
        for node in L.nodes:
            edges = G.edges(node)
            if edges:
                edge_features = G.edges[node]['weight']
                L.nodes[node]['e_f'] = edge_features
            else:
                L.nodes[node]['e_f'] = None
        return L

    def line_graph_node_features(G):
        node_features = []
        for _, node_data in G.nodes(data=True):
            node_features.append(node_data['e_f'])
        return node_features

    def complete_line_graph(L):
        #L = line_graph(G)
        node_features1 = graph_to_LineGraph.line_graph_node_features(L)

        # Create a mapping from original graph edges to line graph nodes
        edge_to_index = {edge: idx for idx, edge in enumerate(L.nodes())}

        # Prepare edge index for the line graph
        edge_index_t = []
        for edge in L.edges:
            src, dst = edge
            edge_index_t.append([edge_to_index[src], edge_to_index[dst]])

        # Convert edge list to a 2D tensor
        edge_index_lg = torch.tensor(edge_index_t, dtype=torch.long).t().contiguous()
        #print(edge_index_lg)
        # Convert node features to tensor
        #nf = torch.tensor(node_features1, dtype=torch.float32).unsqueeze(1)


        #nx.draw(G, with_labels= True)
        # Transform the graph into its line graph representation
       # print('Original Graph: ',G)


        #nx.draw(L, with_labels= True)
        print('Line Graph: ',L)
        return L, edge_index_lg, node_features1
        
        
   
    
    
 
 